In [93]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import pickle
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from typing import List
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from imblearn.under_sampling import RandomUnderSampler 
import collections
from sklearn.metrics import f1_score

In [94]:
df_val = pd.read_csv('Validation-dataset.csv')
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28997 entries, 0 to 28996
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WELL                      28997 non-null  object 
 1   X                         28997 non-null  float64
 2   Y                         28997 non-null  float64
 3   MD                        28997 non-null  float64
 4   GR                        28997 non-null  float64
 5   RT                        28997 non-null  float64
 6   CN                        28997 non-null  float64
 7   DEN                       28997 non-null  float64
 8   DEPOSITIONAL_ENVIRONMENT  28997 non-null  object 
dtypes: float64(7), object(2)
memory usage: 2.0+ MB


In [95]:
print(df_val.isnull().values.any())
df_val.sample()

False


,WELL,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT
7308,Well-X,2.9956,0.0061,1627.865,106.0913,1.4066,0.3182,2.3552,Transitional


In [96]:
df_val.describe()

,X,Y,MD,GR,RT,CN,DEN
count,28997.000000,28997.000000,28997.000000,28997.000000,28997.000000,28997.000000,28997.000000
mean,1.226964,1.750205,1227.801576,84.103308,4.314957,0.337462,2.208479
std,1.178670,1.090184,289.111458,14.677383,2.831058,0.062961,0.135494
min,0.034400,0.006100,645.373000,35.166500,0.008680,0.126370,1.431400
25%,0.034400,0.006100,991.665000,76.238900,3.264980,0.290410,2.113500
50%,1.147500,2.131000,1233.273000,85.604200,3.815280,0.329890,2.201400
75%,2.995600,2.689300,1474.900000,93.001300,4.658960,0.375030,2.322700
max,2.995600,2.689300,1748.073000,142.771900,122.294700,0.699350,2.701900


In [97]:
df = pd.read_csv('Train-dataset.csv')
print(df.isnull().values.any())
df.info()

False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45749 entries, 0 to 45748
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   WELL                      45749 non-null  object 
 1   X                         45749 non-null  float64
 2   Y                         45749 non-null  float64
 3   MD                        45749 non-null  float64
 4   GR                        45749 non-null  float64
 5   RT                        45749 non-null  float64
 6   CN                        45749 non-null  float64
 7   DEN                       45749 non-null  float64
 8   DEPOSITIONAL_ENVIRONMENT  45749 non-null  object 
 9   LITH_NAME                 45749 non-null  object 
 10  LITH_CODE                 45749 non-null  int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 3.8+ MB


In [98]:
df.describe()

,X,Y,MD,GR,RT,CN,DEN,LITH_CODE
count,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000
mean,1.670183,1.428714,1289.498006,84.873762,5.385460,0.345358,2.201688,594.933223
std,1.015506,1.099132,387.339595,15.915062,74.537883,0.076597,0.152785,374.113902
min,0.364000,0.000000,640.983000,25.000000,0.444750,0.062000,1.381500,100.000000
25%,0.756400,0.000000,980.300000,75.904300,3.130500,0.292170,2.098700,400.000000
50%,1.310900,1.983300,1231.900000,86.297800,3.750600,0.333630,2.197695,500.000000
75%,3.000000,2.091200,1562.183000,95.251900,4.652600,0.387720,2.330100,600.000000
max,3.000000,3.000000,2275.600000,146.509000,5000.000000,0.794296,2.725900,1500.000000


In [99]:
df.sample(5)

,WELL,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT,LITH_NAME,LITH_CODE
14326,Well-11,0.7564,1.9833,1323.500,71.51500,5.61900,0.34477,2.15600,Transitional,Sandstone,600
29348,Well-7,1.3109,2.0912,1656.583,109.23180,2.93558,0.29591,2.44820,Marine,Clay marl,400
40890,Well-10,3.0000,0.0000,1789.800,93.98840,2.97968,0.30852,2.40530,Marine,Clay marl,400
24996,Well-7,1.3109,2.0912,1221.383,96.41016,2.93636,0.42694,2.19917,Transitional,Clay,100
44151,Well-10,3.0000,0.0000,2115.900,94.35700,1.95870,0.29069,2.42990,Marine,Clay marl,400


In [100]:
df.isnull().values.any()

False

In [101]:
encoder = {}
decoder = {}
for i, code in enumerate(df['LITH_CODE'].unique()):
    decoder[i] = code
    encoder[code] = i

In [102]:
feature = ['MD','GR', 'RT', 'DEN', 'CN', 'DEPOSITIONAL_ENVIRONMENT']
numeric_features = ['MD','GR', 'RT', 'DEN', 'CN']
categorical_features = ['DEPOSITIONAL_ENVIRONMENT']

In [103]:
X = df[feature]
y = df['LITH_CODE'].apply(lambda x: encoder[x])
X_val = df_val[feature]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [72]:
def data_preprocessing(df: pd.DataFrame,
                       categorical_features: List[str],
                       numeric_features: List[str],
                       scale_strategies) -> pd.DataFrame:
    numeric_transformer = Pipeline(
        steps=[
            ('scaler', scale_strategies)
        ]
    )
    #One-hot encoder
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
    # Preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', numeric_transformer, numeric_features),
            ('categorical', categorical_transformer, categorical_features)
        ]
    )
    return preprocessor

In [ ]:
classifiers = [
    #KNeighborsClassifier(3), 0.834
    #SVC(kernel="rbf", C=0.025, probability=True),0.734
    xgb.XGBClassifier(),
    LGBMClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]
for classifier in classifiers:
    preprocessor = data_preprocessing(X_train,
                                      categorical_features,
                                      numeric_features,
                                      StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
model score: 0.831
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
model score: 0.743
RandomForestClassifier(bootstrap=T

In [ ]:
classifier = RandomForestClassifier()

preprocessor = data_preprocessing(X_train,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])
pipe.fit(X_train, y_train)   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model score: 0.916


In [84]:
classifier = RandomForestClassifier()

preprocessor = data_preprocessing(X_train,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])

param_grid = {
    'model__max_depth': [2, 3, 5, 7, 10],
    'model__n_estimators': [25, 50, 100, 150, 200],
}

cv = GridSearchCV(estimator=pipe, cv=5,
                  param_grid=param_grid)
cv.fit(X_train, y_train)
print(classifier)
print(cv.best_params_)
print("model score: %.3f" % cv.best_estimator_.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
{'model__max_depth': 10, 'model__n_estimators': 150}
model score: 0.848


In [116]:
classifier = RandomForestClassifier(class_weight='balanced')

preprocessor = data_preprocessing(X_train,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])
pipe.fit(X_train, y_train)   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model score: 0.915


In [117]:
preds = pipe.predict(X_val)
preds

array([ 5,  5,  5, ..., 10, 10, 10])

In [118]:
predictions = []
for pred in preds:
    predictions.append(decoder[pred])

In [119]:
np.savetxt('prediction.csv', predictions, delimiter=',', encoding='utf-8') 

In [134]:
X = df[feature]
y = df['LITH_CODE'].apply(lambda x: encoder[x])
X_val = df_val[feature]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [136]:
import collections
count = collections.Counter(y)
count

Counter({0: 889,
         1: 5677,
         2: 11572,
         3: 443,
         4: 83,
         5: 7716,
         6: 340,
         7: 693,
         8: 6496,
         9: 1236,
         10: 8054,
         11: 3,
         12: 2547})

In [160]:
from imblearn.over_sampling import RandomOverSampler

In [181]:
X = df[feature]
y = df['LITH_CODE'].apply(lambda x: encoder[x])
X_val = df_val[feature]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)
# summarize class distribution
print(collections.Counter(y_train))
print(collections.Counter(y_over))
df_over = pd.DataFrame(data = X_over,
                  columns = X.columns.to_list())

Counter({2: 9275, 10: 6403, 5: 6151, 8: 5227, 1: 4572, 12: 2045, 9: 985, 0: 714, 7: 545, 3: 349, 6: 267, 4: 64, 11: 2})
Counter({2: 9275, 11: 9275, 10: 6403, 5: 6151, 8: 5227, 1: 4572, 12: 2045, 9: 985, 0: 714, 7: 545, 3: 349, 6: 267, 4: 64})


In [183]:
classifier = RandomForestClassifier()

preprocessor = data_preprocessing(df_over,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])
pipe.fit(df_over, y_over)   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model score: 0.911


In [220]:
names = y_train.unique()
collections.Counter(y_train)

Counter({0: 714,
         1: 4572,
         2: 9275,
         3: 349,
         4: 64,
         5: 6151,
         6: 267,
         7: 545,
         8: 5227,
         9: 985,
         10: 6403,
         11: 2,
         12: 2045})

In [221]:
max_samples = 985

In [105]:
df_train = X_train.join(y_train)
df_train

,MD,GR,RT,DEN,CN,DEPOSITIONAL_ENVIRONMENT,LITH_CODE
31402,841.000,110.3697,4.20527,2.102200,0.384470,Continental,3
45407,2241.500,81.5272,2.02112,2.419500,0.280210,Marine,2
45074,2208.200,102.2406,3.11045,2.482800,0.273900,Marine,10
43427,2043.500,92.9224,4.05848,2.457500,0.242690,Marine,10
6031,1049.800,83.8033,3.71400,2.200675,0.360977,Continental,8
...,...,...,...,...,...,...,...
4019,848.200,59.6182,6.72760,2.014838,0.394977,Continental,2
31007,801.500,83.9968,4.73844,2.186100,0.345800,Continental,8
21064,828.183,96.7721,3.90289,2.071760,0.443140,Continental,5
30257,726.500,96.2957,5.33277,2.092200,0.364990,Continental,5


In [226]:
new_df = pd.DataFrame(columns = df_train.columns.to_list())
new_df

,MD,GR,RT,DEN,CN,DEPOSITIONAL_ENVIRONMENT,LITH_CODE


In [229]:
df_train['LITH_CODE'].unique()

array([ 2,  8,  5,  1, 10,  9, 12,  3,  7,  0,  6,  4, 11])

In [230]:
for key in df_train['LITH_CODE'].unique():
    if len(df_train[df_train['LITH_CODE'] == key]) < max_samples:
        new_df = new_df.append(df_train[df_train['LITH_CODE'] == key])
    else:
        temp = df_train[df_train['LITH_CODE'] == key].sample(max_samples)
        new_df = new_df.append(temp)

In [231]:
new_df.reset_index(inplace=True)

In [238]:
new_df.drop(columns='index')

,MD,GR,RT,DEN,CN,DEPOSITIONAL_ENVIRONMENT,LITH_CODE
0,1951.400,102.38220,3.35595,2.35010,0.24663,Marine,2
1,893.600,49.51060,6.88370,1.98520,0.35190,Continental,2
2,1029.983,48.32566,7.12176,1.95737,0.27762,Continental,2
3,1494.900,65.06830,3.12140,2.08720,0.27477,Transitional,2
4,1403.900,67.25640,4.31681,2.17660,0.32665,Transitional,2
...,...,...,...,...,...,...,...
8831,1398.000,77.31000,4.95400,2.39000,0.28000,Transitional,4
8832,1149.900,72.87000,3.61800,2.15000,0.42000,Transitional,4
8833,1395.600,78.56000,4.61700,2.37000,0.30000,Transitional,4
8834,1802.500,33.40000,6.22500,2.48000,0.14000,Marine,11


In [286]:
y_new_df = new_df['LITH_CODE']
X_new_df = new_df.drop(columns=['LITH_CODE', 'index'])

In [253]:
classifier = RandomForestClassifier()

preprocessor = data_preprocessing(X_new_df,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])
pipe.fit(X_new_df, y_new_df.astype('int'))   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model score: 0.856


In [254]:
classifier = RandomForestClassifier(class_weight='balanced')

preprocessor = data_preprocessing(X_new_df,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])
pipe.fit(X_new_df, y_new_df.astype('int'))   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model score: 0.854


In [261]:
combined_predictions = []

In [290]:
for j in range(10):
    new_df = pd.DataFrame(columns = df_train.columns.to_list())
    for key in df_train['LITH_CODE'].unique():
        if len(df_train[df_train['LITH_CODE'] == key]) < max_samples:
            new_df = new_df.append(df_train[df_train['LITH_CODE'] == key])
        else:
            temp = df_train[df_train['LITH_CODE'] == key].sample(max_samples, random_state=np.random.randint(1, 200))
            new_df = new_df.append(temp)
    y_new_df = new_df['LITH_CODE']
    X_new_df = new_df.drop(columns=['LITH_CODE'])
    if j % 2 == 0:
        classifier = RandomForestClassifier(class_weight='balanced')
    else:
        classifier = RandomForestClassifier()

    preprocessor = data_preprocessing(X_new_df,
                                      categorical_features,
                                      numeric_features,
                                      StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', classifier)])
    pipe.fit(X_new_df, y_new_df.astype('int'))   
    #print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    combined_predictions.append(pipe.predict(X_val))

Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.852
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.849
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.857
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.853
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.858
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.854
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.854
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
model score: 0.855
Empty DataFrame
Columns: [MD, GR, RT, DEN, CN, DEPOSITIONAL_ENVIRONMENT, LITH_CODE]
Index: []
mo

In [293]:
submission_df = pd.DataFrame({i: combined_predictions[i] for i in range(len(combined_predictions))})
submission_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
16197,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2590,5,5,5,5,5,5,5,5,5,5,5,5,3,5,5,9,5,5,3,5,5,3,3,5,5,5,3,5,5,5,5,5,5,3
13856,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,0,2,2,2,2
18424,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
6218,6,10,10,6,6,10,10,10,10,10,10,6,10,10,6,6,6,10,6,0,6,0,6,10,6,6,10,6,10,6,6,6,6,10


In [152]:
def most_frequent(List):
    occurence_count = collections.Counter(List)
    return occurence_count.most_common(1)[0][0]

In [295]:
sub = []
for row in range(len(submission_df)):
    sub.append(most_frequent(submission_df.loc[row].to_list()))

In [ ]:
predictions = []
for pred in sub:
    predictions.append(decoder[pred])
predictions

In [299]:
np.savetxt('prediction_ensemble.csv', predictions, delimiter=',', encoding='utf-8') 

In [ ]:
# After submitting I have checked and found that there is an outlier in column 'RT'
# I think it would help to boost the score even higher

In [12]:
df.describe()

,X,Y,MD,GR,RT,CN,DEN,LITH_CODE
count,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000,45749.000000
mean,1.670183,1.428714,1289.498006,84.873762,5.385460,0.345358,2.201688,594.933223
std,1.015506,1.099132,387.339595,15.915062,74.537883,0.076597,0.152785,374.113902
min,0.364000,0.000000,640.983000,25.000000,0.444750,0.062000,1.381500,100.000000
25%,0.756400,0.000000,980.300000,75.904300,3.130500,0.292170,2.098700,400.000000
50%,1.310900,1.983300,1231.900000,86.297800,3.750600,0.333630,2.197695,500.000000
75%,3.000000,2.091200,1562.183000,95.251900,4.652600,0.387720,2.330100,600.000000
max,3.000000,3.000000,2275.600000,146.509000,5000.000000,0.794296,2.725900,1500.000000


In [16]:
df[df['RT'] != df['RT'].max()].describe()

,X,Y,MD,GR,RT,CN,DEN,LITH_CODE
count,45740.000000,45740.000000,45740.000000,45740.000000,45740.000000,45740.000000,45740.000000,45740.000000
mean,1.670253,1.428583,1289.624725,84.876400,4.402698,0.345294,2.201757,594.912549
std,1.015593,1.099201,387.272329,15.915355,25.444036,0.076466,0.152706,374.135533
min,0.364000,0.000000,640.983000,25.000000,0.444750,0.062000,1.381500,100.000000
25%,0.756400,0.000000,980.495750,75.917375,3.130470,0.292167,2.098768,400.000000
50%,1.310900,1.983300,1232.000000,86.300915,3.750365,0.333610,2.197700,500.000000
75%,3.000000,2.091200,1562.220750,95.254150,4.651915,0.387665,2.330200,600.000000
max,3.000000,3.000000,2275.600000,146.509000,4748.809300,0.794296,2.725900,1500.000000


In [24]:
df['RT'].sort_values(ascending=False)[10:20]

19232    1997.63924
19243    1621.56418
10250     228.76410
10251     225.03630
1811      161.58300
1812      150.93400
10252     142.46629
10249     135.86700
1810      131.37000
10255     116.31100
Name: RT, dtype: float64

In [25]:
df[df['RT'] < 1600].describe()

,X,Y,MD,GR,RT,CN,DEN,LITH_CODE
count,45737.000000,45737.000000,45737.000000,45737.000000,45737.000000,45737.000000,45737.000000,45737.000000
mean,1.670277,1.428540,1289.666963,84.877239,4.220028,0.345277,2.201774,594.916588
std,1.015622,1.099224,387.249911,15.915316,3.346688,0.076436,0.152691,374.147276
min,0.364000,0.000000,640.983000,25.000000,0.444750,0.062000,1.381500,100.000000
25%,0.756400,0.000000,980.500000,75.917600,3.130410,0.292160,2.098790,400.000000
50%,1.310900,1.983300,1232.000000,86.301300,3.750300,0.333610,2.197740,500.000000
75%,3.000000,2.091200,1562.283000,95.255500,4.651800,0.387650,2.330200,600.000000
max,3.000000,3.000000,2275.600000,146.509000,228.764100,0.794296,2.725900,1500.000000


In [26]:
df_outlier = df[df['RT'] < 1600]

In [34]:
collections.Counter(df_outlier['LITH_CODE'])

Counter({100: 7716,
         200: 3,
         300: 1236,
         400: 8054,
         500: 6491,
         600: 11568,
         800: 693,
         1000: 2544,
         1100: 443,
         1200: 889,
         1300: 5677,
         1400: 83,
         1500: 340})

In [43]:
X = df_outlier[feature]
y = df_outlier['LITH_CODE'].apply(lambda x: encoder[x])
X_val = df_val[feature]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [44]:
collections.Counter(y_train)

Counter({0: 714,
         1: 4517,
         2: 9227,
         3: 358,
         4: 64,
         5: 6139,
         6: 276,
         7: 561,
         8: 5197,
         9: 999,
         10: 6488,
         11: 2,
         12: 2047})

In [45]:
collections.Counter(y_test)

Counter({0: 175,
         1: 1160,
         2: 2341,
         3: 85,
         4: 19,
         5: 1577,
         6: 64,
         7: 132,
         8: 1294,
         9: 237,
         10: 1566,
         11: 1,
         12: 497})

In [49]:
from sklearn.metrics import f1_score
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf"),
    xgb.XGBClassifier(),
    LGBMClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]
for classifier in classifiers:
    preprocessor = data_preprocessing(X_train,
                                      categorical_features,
                                      numeric_features,
                                      StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
model score: 0.860
F1 score: 0.860
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
model score: 0.794
F1 score: 0.769
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
model score: 0.833
F1 sc

In [51]:
from sklearn.metrics import f1_score
classifiers = RandomForestClassifier(max_depth=10,
                                     n_estimators=150)
preprocessor = data_preprocessing(X_train,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])
pipe.fit(X_train, y_train)   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
model score: 0.860
F1 score: 0.850


In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf"),
    xgb.XGBClassifier(),
    LGBMClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]
for classifier in classifiers:
    preprocessor = data_preprocessing(X_train,
                                      categorical_features,
                                      numeric_features,
                                      StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))

In [58]:
classifier = xgb.XGBClassifier()

preprocessor = data_preprocessing(X_train,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])

param_grid = {
    'model__eta': [0.01, 0.1, 0.2, 0.3],
    'model__min_child_weight': [1, 2, 3],
    #'model__max_depth': np.arange(2, 11, 1),
    #'model__subsample': np.arange(0.5, 1.1, 0.1),
    #'model__colsample_bytree': np.arange(0.5, 1.1, 0.1)
}

cv = GridSearchCV(estimator=pipe, cv=5,
                  param_grid=param_grid)
cv.fit(X_train, y_train)
print(classifier)
print(cv.best_params_)
print("model score: %.3f" % cv.best_estimator_.score(X_test, y_test))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
{'model__eta': 0.01, 'model__min_child_weight': 2}
model score: 0.834


In [60]:
print("F1 score: %.3f" % f1_score(y_test, cv.best_estimator_.predict(X_test), average='weighted'))

F1 score: 0.817


In [61]:
classifier = xgb.XGBClassifier()

preprocessor = data_preprocessing(X_train,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])

param_grid = {
    'model__eta': [0.01],
    'model__min_child_weight': [2],
    'model__max_depth': np.arange(2, 11, 1),
    #'model__subsample': np.arange(0.5, 1.1, 0.1),
    #'model__colsample_bytree': np.arange(0.5, 1.1, 0.1)
}

cv = GridSearchCV(estimator=pipe, cv=5,
                  param_grid=param_grid)
cv.fit(X_train, y_train)
print(classifier)
print(cv.best_params_)
print("model score: %.3f" % cv.best_estimator_.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, cv.best_estimator_.predict(X_test), average='weighted'))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
{'model__eta': 0.01, 'model__max_depth': 10, 'model__min_child_weight': 2}
model score: 0.923
F1 score: 0.922


In [62]:
classifier = xgb.XGBClassifier()

preprocessor = data_preprocessing(X_train,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])

param_grid = {
    'model__eta': [0.01],
    'model__min_child_weight': [2],
    'model__max_depth': [10],
    'model__subsample': np.arange(0.5, 1.1, 0.1),
    #'model__colsample_bytree': np.arange(0.5, 1.1, 0.1)
}

cv = GridSearchCV(estimator=pipe, cv=5,
                  param_grid=param_grid)
cv.fit(X_train, y_train)
print(classifier)
print(cv.best_params_)
print("model score: %.3f" % cv.best_estimator_.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, cv.best_estimator_.predict(X_test), average='weighted'))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
{'model__eta': 0.01, 'model__max_depth': 10, 'model__min_child_weight': 2, 'model__subsample': 0.7999999999999999}
model score: 0.926
F1 score: 0.924


In [63]:
classifier = xgb.XGBClassifier()

preprocessor = data_preprocessing(X_train,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])

param_grid = {
    'model__eta': [0.01],
    'model__min_child_weight': [2],
    'model__max_depth': [10],
    'model__subsample': [0.8],
    'model__colsample_bytree': 1
}

cv = GridSearchCV(estimator=pipe, cv=5,
                  param_grid=param_grid)
cv.fit(X_train, y_train)
print(classifier)
print(cv.best_params_)
print("model score: %.3f" % cv.best_estimator_.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, cv.best_estimator_.predict(X_test), average='weighted'))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
{'model__colsample_bytree': 0.9999999999999999, 'model__eta': 0.01, 'model__max_depth': 10, 'model__min_child_weight': 2, 'model__subsample': 0.8}
model score: 0.926
F1 score: 0.924


In [65]:
classifier = xgb.XGBClassifier(
    eta=0.01,
    min_child_weight=2,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=1
)

preprocessor = data_preprocessing(X_train,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])


pipe.fit(X_train, y_train)
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
model score: 0.926
F1 score: 0.924


In [64]:
classifier = xgb.XGBClassifier(
    eta=0.01,
    min_child_weight=2,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=1
)

preprocessor = data_preprocessing(X_train,
                                  categorical_features,
                                  numeric_features,
                                  StandardScaler())
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])


pipe.fit(X_train, y_train, model__sample_weight=classes_weights)
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
model score: 0.918
F1 score: 0.919


In [56]:
from sklearn.utils import class_weight
classifiers = xgb.XGBClassifier()

preprocessor = data_preprocessing(X_train,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', classifier)])

classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

pipe.fit(X_train, y_train, model__sample_weight=classes_weights)   
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))
print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
model score: 0.821
F1 score: 0.828


In [132]:
max_samples = 1000
for j in range(10):
    new_df = pd.DataFrame(columns = df_train.columns.to_list())
    for key in df_train['LITH_CODE'].unique():
        if len(df_train[df_train['LITH_CODE'] == key]) < max_samples:
            new_df = new_df.append(df_train[df_train['LITH_CODE'] == key])
        else:
            temp = df_train[df_train['LITH_CODE'] == key].sample(max_samples, random_state=np.random.randint(1, 200))
            new_df = new_df.append(temp)
    y_new_df = new_df['LITH_CODE']
    X_new_df = new_df.drop(columns=['LITH_CODE'])
    classifier = xgb.XGBClassifier(
        eta=0.01,
        min_child_weight=2,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=1
    )

    preprocessor = data_preprocessing(X_new_df,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', classifier)])


    pipe.fit(X_new_df, y_new_df.astype('int'))
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))
    #combined_predictions.append(pipe.predict(X_val))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
model score: 0.870
F1 score: 0.876
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
m

In [133]:
max_samples = 3000
for j in range(10):
    new_df = pd.DataFrame(columns = df_train.columns.to_list())
    for key in df_train['LITH_CODE'].unique():
        if len(df_train[df_train['LITH_CODE'] == key]) < max_samples:
            new_df = new_df.append(df_train[df_train['LITH_CODE'] == key])
        else:
            temp = df_train[df_train['LITH_CODE'] == key].sample(max_samples, random_state=np.random.randint(1, 200))
            new_df = new_df.append(temp)
    y_new_df = new_df['LITH_CODE']
    X_new_df = new_df.drop(columns=['LITH_CODE'])
    classifier = xgb.XGBClassifier(
        eta=0.01,
        min_child_weight=2,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=1,
        objective='auc'
    )

    preprocessor = data_preprocessing(X_new_df,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', classifier)])


    pipe.fit(X_new_df, y_new_df.astype('int'))
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))
    #combined_predictions.append(pipe.predict(X_val))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
model score: 0.915
F1 score: 0.915
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
m

In [136]:
f1_score(y_new_df.astype('int'), pipe.predict(X_new_df), average='weighted')

0.9909067095828835

In [139]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [141]:
kfold = StratifiedKFold(n_splits=10, random_state=7)
cross_val_score(pipe, X_new_df, y_new_df.astype('int'), cv=kfold)

array([0.89476309, 0.89177057, 0.9032419 , 0.89526185, 0.8872818 ,
       0.88678304, 0.8882793 , 0.89226933, 0.89326683, 0.90374065])

In [143]:
kfold = StratifiedKFold(n_splits=10, random_state=7)
cross_val_score(pipe, df[feature], df['LITH_CODE'], cv=kfold)

array([0.68437158, 0.75387978, 0.75912568, 0.69355191, 0.69879781,
       0.73704918, 0.68284153, 0.85464481, 0.85464481, 0.51268037])

In [144]:
kfold = StratifiedKFold(n_splits=10, random_state=7)
cross_val_score(pipe, df_outlier[feature], df_outlier['LITH_CODE'], cv=kfold)

array([0.68320944, 0.75120245, 0.76038478, 0.69348491, 0.70332313,
       0.73874071, 0.68495846, 0.85458124, 0.85392521, 0.53269189])

In [146]:
max_samples = 3500
for j in range(10):
    new_df = pd.DataFrame(columns = df_train.columns.to_list())
    for key in df_train['LITH_CODE'].unique():
        if len(df_train[df_train['LITH_CODE'] == key]) < max_samples:
            new_df = new_df.append(df_train[df_train['LITH_CODE'] == key])
        else:
            temp = df_train[df_train['LITH_CODE'] == key].sample(max_samples, random_state=np.random.randint(1, 200))
            new_df = new_df.append(temp)
    y_new_df = new_df['LITH_CODE']
    X_new_df = new_df.drop(columns=['LITH_CODE'])
    classifier = xgb.XGBClassifier(
        eta=0.01,
        min_child_weight=2,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=1,
        objective='auc'
    )

    preprocessor = data_preprocessing(X_new_df,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', classifier)])


    pipe.fit(X_new_df, y_new_df.astype('int'))
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    print("F1 score: %.3f" % f1_score(y_test, pipe.predict(X_test), average='weighted'))
    #combined_predictions.append(pipe.predict(X_val))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
model score: 0.920
F1 score: 0.920
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)
m

In [147]:
combined_predictions = {}
max_samples = 3500
acc = {}
for j in range(10):
    new_df = pd.DataFrame(columns = df_train.columns.to_list())
    for key in df_train['LITH_CODE'].unique():
        if len(df_train[df_train['LITH_CODE'] == key]) < max_samples:
            new_df = new_df.append(df_train[df_train['LITH_CODE'] == key])
        else:
            temp = df_train[df_train['LITH_CODE'] == key].sample(max_samples, random_state=np.random.randint(1, 200))
            new_df = new_df.append(temp)
    y_new_df = new_df['LITH_CODE']
    X_new_df = new_df.drop(columns=['LITH_CODE'])
    classifier = xgb.XGBClassifier(
        eta=0.01,
        min_child_weight=2,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=1,
        objective='auc'
    )

    preprocessor = data_preprocessing(X_new_df,
                                    categorical_features,
                                    numeric_features,
                                    StandardScaler())

    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', classifier)])


    pipe.fit(X_new_df, y_new_df.astype('int'))
    #print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    f1 = f1_score(y_test, pipe.predict(X_test), average='weighted')
    print("F1 score: %.3f" % f1)
    acc[j] = f1
    combined_predictions[j] = pipe.predict(X_val)

model score: 0.920
F1 score: 0.920
model score: 0.921
F1 score: 0.921
model score: 0.917
F1 score: 0.917
model score: 0.916
F1 score: 0.917
model score: 0.920
F1 score: 0.920
model score: 0.916
F1 score: 0.916
model score: 0.915
F1 score: 0.915
model score: 0.917
F1 score: 0.917
model score: 0.920
F1 score: 0.920
model score: 0.918
F1 score: 0.918


In [148]:
acc

{0: 0.9198034131319924,
 1: 0.9205901715831966,
 2: 0.917066191116951,
 3: 0.9165276236568299,
 4: 0.9195801455247599,
 5: 0.9159129614037008,
 6: 0.9152640521922015,
 7: 0.9171524200870815,
 8: 0.9197868193311758,
 9: 0.9176313977853888}

In [150]:
array = [0, 1, 4, 8, 9]
submission_df = pd.DataFrame({idx: combined_predictions[i] for idx, i in enumerate(array)})
submission_df.sample(5)

,0,1,2,3,4
20470,12,12,12,12,12
10335,8,8,8,8,8
15668,2,2,2,2,2
18941,3,3,3,5,3
14988,2,2,2,2,2


In [ ]:
sub = []
for row in range(len(submission_df)):
    sub.append(most_frequent(submission_df.loc[row].to_list()))
predictions = []
for pred in sub:
    predictions.append(decoder[pred])
predictions

In [154]:
predictions = []
for pred in sub:
    predictions.append(decoder[pred])
predictions
np.savetxt('prediction_xgb.csv', predictions, delimiter=',', encoding='utf-8') 